# More messing with grammar

In [1]:
import pyparsing as pp
from eisenscript_to_xml import eisenscript_to_xml as ex

In [2]:
bcall = '{ x 1 y 1 s 1 } sphere'

In [3]:
fnum = pp.Word(".+-*/()" + pp.nums)
loop_multiplier = fnum("count") + pp.Suppress("*")


tid = pp.oneOf("x y z s rx ry rz", caseless=True)("tid")
# these are color transforms that we're ignoring
cid = pp.oneOf("h hue sat b brightness a alpha m", caseless=True)
tvalues = pp.OneOrMore(fnum)("tvalues")
gtrans = pp.Group(tid + tvalues).setResultsName("trans", listAllMatches=True)
ctrans = cid + tvalues
c2trans = pp.CaselessKeyword("color") + pp.Word(pp.alphanums + "#")
c3trans = pp.CaselessKeyword("blend") + pp.Word(pp.alphanums + "#") + fnum
trans = gtrans | ctrans | c2trans | c3trans

loop = pp.Group(
    pp.Optional(loop_multiplier)
    + pp.Suppress("{")
    + pp.ZeroOrMore(trans)
    + pp.Suppress("}")
).setResultsName("loop", listAllMatches=True)

shape_words = pp.oneOf(["box", "grid", "sphere", "line"], caseless=True)
shape = pp.Combine(shape_words + pp.Optional(pp.Word(pp.alphas + ":")))

shape_call = (pp.Optional(loop) + shape("shape")).setResultsName(
        "bcall", listAllMatches=True
    )
t = shape_call.parse_string(bcall)
t

ParseResults([ParseResults([ParseResults(['x', '1'], {'tid': 'x', 'tvalues': ['1']}), ParseResults(['y', '1'], {'tid': 'y', 'tvalues': ['1']}), ParseResults(['s', '1'], {'tid': 's', 'tvalues': ['1']})], {'trans': [{'tid': 'x', 'tvalues': ['1']}, {'tid': 'y', 'tvalues': ['1']}, {'tid': 's', 'tvalues': ['1']}]}), 'sphere'], {'loop': {'trans': [{'tid': 'x', 'tvalues': ['1']}, {'tid': 'y', 'tvalues': ['1']}, {'tid': 's', 'tvalues': ['1']}]}, 'shape': 'sphere', 'bcall': [[{'trans': [{'tid': 'x', 'tvalues': ['1']}, {'tid': 'y', 'tvalues': ['1']}, {'tid': 's', 'tvalues': ['1']}]}, 'sphere']]})

In [4]:
t.as_list()

[[['x', '1'], ['y', '1'], ['s', '1']], 'sphere']

In [5]:
t.as_dict()

{'loop': {'trans': [{'tid': 'x', 'tvalues': ['1']},
   {'tid': 'y', 'tvalues': ['1']},
   {'tid': 's', 'tvalues': ['1']}]},
 'shape': 'sphere',
 'bcall': [[{'trans': [{'tid': 'x', 'tvalues': ['1']},
     {'tid': 'y', 'tvalues': ['1']},
     {'tid': 's', 'tvalues': ['1']}]},
   'sphere']]}

what I'd like is just - or some such
{'bcall': [{'trans': [{'tid': 'x', 'tvalues': ['1']},
     {'tid': 'y', 'tvalues': ['1']},
     {'tid': 's', 'tvalues': ['1']}]},
   'shape': 'sphere']]}

In [67]:
fnum = pp.Word(".+-*/()" + pp.nums)
loop_multiplier = fnum("count") + pp.Suppress("*")


tid = pp.oneOf("x y z s rx ry rz", caseless=True)("tid")
# these are color transforms that we're ignoring
cid = pp.oneOf("h hue sat b brightness a alpha m", caseless=True)
tvalues = pp.OneOrMore(fnum)("tvalues")
gtrans = pp.Group(tid + tvalues)
ctrans = cid + tvalues
c2trans = pp.CaselessKeyword("color") + pp.Word(pp.alphanums + "#")
c3trans = pp.CaselessKeyword("blend") + pp.Word(pp.alphanums + "#") + fnum
trans = gtrans | pp.Suppress(ctrans) | pp.Suppress(c2trans) | pp.Suppress(c3trans)

loop = (
    pp.Optional(loop_multiplier)
    + pp.Suppress("{")
    + pp.ZeroOrMore(trans)
    + pp.Suppress("}")
)

shape_words = pp.oneOf(["box", "grid", "sphere", "line"], caseless=True)
shape = pp.Combine(shape_words + pp.Optional(pp.Word(pp.alphas + ":")))

shape_call = (pp.Optional(loop)("trans") + shape("shape"))

t = shape_call.parse_string(bcall)
t

ParseResults([ParseResults(['x', '1'], {'tid': 'x', 'tvalues': ['1']}), ParseResults(['y', '1'], {'tid': 'y', 'tvalues': ['1']}), ParseResults(['s', '1'], {'tid': 's', 'tvalues': ['1']}), 'sphere'], {'trans': [{'tid': 'x', 'tvalues': ['1']}, {'tid': 'y', 'tvalues': ['1']}, {'tid': 's', 'tvalues': ['1']}], 'shape': 'sphere'})

In [68]:
t.as_list()

[['x', '1'], ['y', '1'], ['s', '1'], 'sphere']

In [69]:
t.as_dict()

{'trans': [{'tid': 'x', 'tvalues': ['1']},
  {'tid': 'y', 'tvalues': ['1']},
  {'tid': 's', 'tvalues': ['1']}],
 'shape': 'sphere'}

In [70]:
t = shape_call.parse_string('{ hue 113 sat 19 a 23 s 0.1 0.9 1.1 } box')
t.as_dict()

{'trans': [{'tid': 's', 'tvalues': ['0.1', '0.9', '1.1']}], 'shape': 'box'}

In [71]:
tstr = '{  x 30 y 30 z -3 s 900 900 1 color white } box'
t = shape_call.parse_string(tstr)
t.as_dict()

{'trans': [{'tid': 'x', 'tvalues': ['30']},
  {'tid': 'y', 'tvalues': ['30']},
  {'tid': 'z', 'tvalues': ['-3']},
  {'tid': 's', 'tvalues': ['900', '900', '1']}],
 'shape': 'box'}

this is good  
do the same tidy up with `rule_call`

In [77]:
rule_name = pp.NotAny(pp.CaselessKeyword("rule")) + pp.Word(
        pp.alphas, pp.alphanums + "_"
    )
rule_call = (pp.Optional(loop)("trans") + rule_name("rule_name"))


In [78]:
tstr = '{ rx 180 x 20  rz 2 } br'
t = rule_call.parse_string(tstr)
t.as_dict()

{'trans': [{'tid': 'rx', 'tvalues': ['180']},
  {'tid': 'x', 'tvalues': ['20']},
  {'tid': 'rz', 'tvalues': ['2']}],
 'rule_name': ['br']}

why the list enclosing `rule_name` but not `shape`? NotAny?,
the difference between a shape call and a rule call is just that shape calls have to use one of the shape words? I think the word rule has to occur at the start of a line so don't need the not any if we assume we have correct eisenscript.

In [79]:
rule_name =  pp.Word(
        pp.alphas, pp.alphanums + "_"
    )
rule_call = (pp.Optional(loop)("trans") + rule_name("rule_name"))

In [80]:
tstr = '{ rx 180 x 20  rz 2 } br'
t = rule_call.parse_string(tstr)
t.as_dict()

{'trans': [{'tid': 'rx', 'tvalues': ['180']},
  {'tid': 'x', 'tvalues': ['20']},
  {'tid': 'rz', 'tvalues': ['2']}],
 'rule_name': 'br'}

In [81]:
call = shape_call | rule_call


In [82]:
tstr = '{ rx 180 x 20  rz 2 } br'
t = call.parse_string(tstr)
t.as_dict()

{'trans': [{'tid': 'rx', 'tvalues': ['180']},
  {'tid': 'x', 'tvalues': ['20']},
  {'tid': 'rz', 'tvalues': ['2']}],
 'rule_name': 'br'}

In [83]:
tstr = '{  x 30 y 30 z -3 s 900 900 1 color white } box'
t = call.parse_string(tstr)
t.as_dict()

{'trans': [{'tid': 'x', 'tvalues': ['30']},
  {'tid': 'y', 'tvalues': ['30']},
  {'tid': 'z', 'tvalues': ['-3']},
  {'tid': 's', 'tvalues': ['900', '900', '1']}],
 'shape': 'box'}

this looks really promising, can we add a field `type` to parseResults? Or just use 

```
if 'shape' in rules:  
```

In [89]:
# added to module
t = ex.rule_shape_grammar().parse_string(tstr)
t.as_dict()


{'trans': [{'tid': 'x', 'tvalues': ['30']},
  {'tid': 'y', 'tvalues': ['30']},
  {'tid': 'z', 'tvalues': ['-3']},
  {'tid': 's', 'tvalues': ['900', '900', '1']}],
 'shape': 'box'}

In [90]:
tstr = '{ rx 180 x 20  rz 2 } br'
t = ex.rule_shape_grammar().parse_string(tstr)
t.as_dict()

{'trans': [{'tid': 'rx', 'tvalues': ['180']},
  {'tid': 'x', 'tvalues': ['20']},
  {'tid': 'rz', 'tvalues': ['2']}],
 'rule_name': 'br'}

The rests of `eisen_grammar` will need a similar treatment.